In [ ]:
import os
import re

import pandas as pd
import numpy as np

input_folder = '../.data' # Where Bin#.csv files are

barcode_column = 'barcode'
tag_column = 'found'
aa_column  = 'aa_mutation'
n_aa_column = 'n_aa_substitutions'
valid_barcode_tag = '1'

filter_freq = 1 # barcodes found more than this.

In [ ]:
for f in os.listdir(input_folder):
    if re.search("^Bin[0-9]+.csv$", f):

        bint = pd.read_csv(os.path.join(input_folder,f))

        tags_sorted = np.sort ( bint[tag_column].unique() )

        #bint_freq = bint.assign(freq=lambda x: x.barcode.map(bint[barcode_column].value_counts())).query(f"freq > {filter_freq}").copy()
        bint_size = bint.groupby(bint.columns.tolist(),as_index=False, dropna=False).size().copy()
        bint_freq = bint_size.query(f"size > {filter_freq}").copy()
        
        bint_freq.to_csv('bint_freq.csv',index=False)
        
        tag_sizes = pd.DataFrame(data={'tag' : ['total'],
                      'n_barcodes': [bint.shape[0]],
                      'n_unique_barcodes': [bint_size.shape[0]],
                      'n_frequent_barcodes': [bint_freq['size'].sum()],
                      'n_unique_frequent_barcodes': [bint_freq.shape[0]]
                     })


        for i in tags_sorted:

            bint_tag = bint.query(f"{tag_column} == {i}")
            #bint_freq = bint_tag.assign(freq=lambda x: x.barcode.map(bint[barcode_column].value_counts())).query(f"freq > {filter_freq}").copy()
            bint_size = bint_tag.groupby(bint.columns.tolist(),as_index=False, dropna=False).size().copy()
            bint_freq = bint_size.query(f"size > {filter_freq}").copy()
            
            tag_sizes = tag_sizes.append(pd.DataFrame(data={'tag' : [i],
                          'n_barcodes': [bint_tag.shape[0]],
                          'n_unique_barcodes': [bint_size.shape[0]],
                          'n_frequent_barcodes': [bint_freq['size'].sum()],
                          'n_unique_frequent_barcodes': [bint_freq.shape[0]]
                         }))

            if str(i) == valid_barcode_tag:
                bint_freq.drop_duplicates()[[aa_column,n_aa_column,'size']].fillna('wt').groupby([aa_column,n_aa_column], as_index=False).sum().to_csv('a_'+f, index=False)

                mutations_sorted = np.sort ( bint_tag[n_aa_column].unique() )
                mutation_sizes =  pd.DataFrame()

                for j in mutations_sorted:
                    mutation_sizes = mutation_sizes.append(pd.DataFrame(data={'mutation': [j],
                        'n_mutations' : [bint_tag.query(f"{n_aa_column} == {j}").shape[0]],
                        'n_unique_mutaions' : [bint_tag.query(f"{n_aa_column} == {j}")[aa_column].unique().shape[0]],
                        'n_frequent_mutations' : [bint_freq.query(f"{n_aa_column} == {j}").shape[0]],
                        'n_unique_frequent_mutations' : [bint_freq.query(f"{n_aa_column} == {j}")[aa_column].unique().shape[0]]
                    }
                    ))


        tag_sizes.index.name = f
        display(tag_sizes)
        mutation_sizes.index.name = f
        display(mutation_sizes)